In [2]:
%matplotlib inline
import numpy as np
#from sklearn.manifold import TSNE
import os

import sys
from pathlib import Path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor
from math import floor
import random
from torch.utils.data import  Dataset
from PIL import Image
from pathlib import Path
import pandas
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torchvision.transforms as transforms

import sys
from pathlib import Path


In [9]:
from pl_models.DEA import DAE_LitModel


pl_model = DAE_LitModel.load_from_checkpoint("/media/data/robert/code/DiffAE/lightning_logs/DAE_NAKO_256/version_1/checkpoints/last.ckpt")

Model params: 15.87 M


In [4]:
# path = "/media/data/robert/code/nako_embedding/dataset/data/train/1004/100400_HWS_07.png"

# from dataloader.dataset_factory import get_dataset, get_data_loader

# val_data = get_dataset(pl_model.conf, split="val")
#

In [5]:
# with torch.no_grad():
#    i: Tensor = val_data[0]["img"]
#    i = i.unsqueeze(0).cuda()
#    i.shape
#    emb = pl_model.model.encode(i).cpu()

In [6]:
# emb.shape

In [7]:
def dict_creation(daten, label):
    occurences = daten[label].to_list()
    occurences.sort()
    occurences = list(dict.fromkeys(occurences))
    label_dict = {}
    for i in range(len(occurences)):
        ky = occurences[i]
        label_dict[ky] = i
    return label_dict


from pathlib import Path

root = "/media/data/robert/code/nako_embedding/"

def_dir = Path(root) / "dataset/vibe/split.csv"


class EnocderSet(Dataset):
    def __init__(
        self,
        directory=def_dir,
        # dorsal=None,
        path=None,
        drop_mean=False,
        transform=None,
        region_of_interest="random",
        slice="center",
        target_transform=None,
        target_label=["PatientSex", "PatientAge", "PatientWeight", "PatientSize"],
        # ind=-150,
        # three_D=None,
        split="all",  # "train",
        return_row=False,
    ):
        super(EnocderSet, self).__init__()

        """
        The target label can be switched to:
        PatientSex
        PatientAge
        PatientSize
        PatientWeight
        """
        # load the .csv which was created previously

        # self.three_D = three_D
        # self.dorsal = dorsal
        self.path = path
        self.target_label = target_label
        self.directory = directory
        # self.ind = ind
        self.name = "StyleGAN2-Dataset"
        self.drop_mean = drop_mean
        self.region_of_interest = region_of_interest
        self.slice = slice
        dataframe = pd.DataFrame(pd.read_csv(directory))

        if split != "all":
            dataframe = pd.DataFrame(pd.read_csv(directory))
            self.dataset = dataframe.loc[dataframe["Split"] > split]
            self.dataset.reset_index(inplace=True)
        else:
            self.dataset = dataframe
        if drop_mean == True:
            self.dataset = self.dataset.drop(self.dataset[(self.dataset["PatientAge"] > 27) & (self.dataset["PatientAge"] < 37)].index)
            self.dataset.reset_index(inplace=True)
        # self.dataset = pd.read_csv(directory)[:ind]

        print("len = ", len(self.dataset))

        self.age_count = self.dataset["PatientAge"].nunique()
        self.weight_count = self.dataset["PatientWeight"].nunique()
        self.size_count = self.dataset["PatientSize"].nunique()
        self.transform = transform
        self.target_transform = target_transform
        self.return_row = return_row

        length_dict = {
            "PatientSex": 2,
            "PatientAge": self.age_count,
            "PatientWeight": self.weight_count,
            "PatientSize": self.size_count,
        }

        self.ind = [2, self.age_count, self.weight_count, self.size_count]

        # for keys in length_dict.keys():
        #    print(keys + " dictionary is ", length_dict[keys], " long")
        self.num_img = {}

    def __len__(self):
        return len(self.dataset)

    def get_max_files(self, index, roi):
        row = self.dataset.iloc[index]
        id = f'{row["PatientID"]}_{roi}'
        if id not in self.num_img:
            p2 = Path(root, "dataset/data/", row["Split"], str(row["PatientID"])[:4])
            self.num_img[id] = len(list(p2.glob(id + "_*.png")))
        return self.num_img[id]

    def get_item(self, index, slice=None, roi=None):
        row = self.dataset.iloc[index]
        if roi is None:
            roi = self.region_of_interest
        if roi == "random":
            roi = ["LWS", "HWS", "BWS"][random.randint(0, 2)]

        p2 = Path(root, "dataset/data/", row["Split"], str(row["PatientID"])[:4])
        if slice is not None:
            i = slice
            if self.get_max_files(index, roi) >= i:
                return None, []
        elif self.slice == "random":
            i = random.randint(0, self.get_max_files(index, roi) - 1)
        elif self.slice == "center":
            i = self.get_max_files(index, roi) // 2
        else:
            i = self.slice  # TODO
        im_path = Path(p2, f'{row["PatientID"]}_{roi}_{i:02}.png')
        # print(im_path)
        if not im_path.exists():
            raise FileNotFoundError(im_path)

        img_array = Image.open(im_path)
        img_array = img_array.convert("L")
        if self.transform:
            img_array = self.transform(img_array)

        if self.return_row:
            return img_array, row
        target = []
        if type(self.target_label) == str:
            target = self.dataset.loc[index, self.target_label]
        elif type(self.target_label) == list:
            for i, label in enumerate(self.target_label):
                target.append(self.dataset.loc[index, label])

        return img_array, target

    def __getitem__(self, index):
        return self.get_item(index)

In [10]:
"""
We found 11189 patients in the NAKO dataset where all three regions exist without duplicate. We initially split the dataset patient wise into train/val/test set 80/5/15
We generate with the encoding an embedding for every image. We use the center crop for every image.

We concatenate 12 slice to improve our T-SNE plots
"""


class ROWS:
    PatientID: str


from pl_models.DEA import DAE_LitModel

pd_org = pandas.read_csv(root + "dataset/split.csv")
checkpoint_path = "/media/data/robert/code/DiffAE/lightning_logs/DAE_NAKO_256/version_1/checkpoints/last.ckpt"
device: str | None = "cuda:0"
print("checkpoint_path", checkpoint_path)
assert Path(checkpoint_path).exists()

print("[1] Reload Encoder4Editing")
encoder = DAE_LitModel.load_from_checkpoint(checkpoint_path)
print("[2] load dataset")
transformation = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(256), transforms.Normalize([0.5], [0.5])])
encoder_set = EnocderSet(transform=transformation, return_row=True)
# loader = DataLoader(encoder_set, batch_size=4)
print("[3] encode dataset")


def encode(encoder_set: EnocderSet, id, roi):
    real_images, label = encoder_set.get_item(id, slice=None, roi=roi)  # type: ignore
    label: ROWS
    real_images = torch.as_tensor(real_images, device=device).unsqueeze_(0)  # type: ignore
    real_images = real_images.to(device)
    latent_code: torch.Tensor = pl_model.model.encode(real_images).to(device)  #
    return latent_code, label


def encode_multi(encoder_set: EnocderSet, id, roi):
    offset = (encoder_set.get_max_files(id, roi) - 12) // 2
    label: ROWS = None  # type: ignore
    latent_codes = []
    for i in range(offset, offset + 12):
        real_images, label = encoder_set.get_item(id, slice=None, roi=roi)  # type: ignore
        real_images = torch.as_tensor(real_images, device=device).unsqueeze_(0)  # type: ignore
        real_images = real_images.to(device)
        latent_code: torch.Tensor = pl_model.model.encode(real_images).to(device)  #
        latent_codes.append(latent_code)
    return torch.cat(latent_codes), label


latent_code, label = encode(encoder_set, 0, "BWS")

checkpoint_path /media/data/robert/code/DiffAE/lightning_logs/DAE_NAKO_256/version_1/checkpoints/last.ckpt
[1] Reload Encoder4Editing
Model params: 15.87 M
[2] load dataset
len =  10823
[3] encode dataset


In [11]:
print("[4] encode dataset")

encoder_set = EnocderSet(transform=transformation, return_row=True, region_of_interest="HWS", slice="center")
dic = {}
l = len(encoder_set)
slice = "DEA_vibe"
file = root + f"/dataset/embeddings_{slice}_stack.pkl"

import pickle

if Path(file).exists():
    with open(file, "rb") as f:
        dic = pickle.load(f)
        print(len(dic))

count = {k: 0 for k in range(20)}
with torch.no_grad():
    for i in range(l):
        count[encoder_set.get_max_files(i, "LWS")] += 1
        try:
            latent_code1, label1 = encode_multi(encoder_set, i, "HWS")  #
            # print(label.PatientID)
            if label1.PatientID in dic:
                continue
            latent_code2, label2 = encode_multi(encoder_set, i, "BWS")
            latent_code3, label3 = encode_multi(encoder_set, i, "LWS")
            assert label1.PatientID == label2.PatientID
            assert label1.PatientID == label3.PatientID
            dic[label1.PatientID] = (label1, latent_code1.cpu(), latent_code2.cpu(), latent_code3.cpu())
            print(f"[5] {i:6} / {l:6}                                             ", end="\r")
            # assert i + 1 == len(dic), (i + 1, len(dic))
            if i % 1001 == 1000:
                with open(file, "wb") as f:
                    pickle.dump(dic, f)
        except Exception as e:
            # raise e
            print(e)
with open(file, "wb") as f:
    pickle.dump(dic, f)

[4] encode dataset
len =  10823


KeyboardInterrupt: 

In [ ]:
extended_list = []

print("[5] encode dataset")
p = Path(root, f"dataset/vibe/train_DAE.csv")
print(p)
data = []
mapping = {"BWS": "thoracic", "HWS": "cervical", "LWS": "lumbar"}
for k in dic.keys():
    print(k, end="\r")
    for roi in ["BWS", "HWS", "LWS"]:
        row = dic[k][0].copy()
        id = f'{row["PatientID"]}_{roi}'
        p2 = Path(root, "dataset/data/", row["Split"], str(row["PatientID"])[:4])
        v = len(list(p2.glob(id + "_*.png")))
        for i in range(v):
            row = dic[k][0].copy()
            p2 = Path(root, "dataset/data/", row["Split"], str(row["PatientID"])[:4])
            row["file_path"] = str(Path(p2, f'{row["PatientID"]}_{roi}_{i:02}.png'))
            row["roi"] = mapping[roi]

            data.append(row.copy())
df = pd.DataFrame(data)
df.to_csv(p)
# df = pd.concat(data, columns=["PatientID"])
# print(df)
print(len(df))


print(count)